In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('dataset/clustered.csv')
test

,id,query,language,cluster
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",thai,0
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...,english,0
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...,english,-1
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",thai,0
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w...",english,1
...,...,...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...,english,0
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D...",thai,0
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ...",thai,0
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...,english,1


In [3]:
system_prompt_mcq = (
    """🧠 系统提示词（含反思与修正步骤）

你是一位严谨的学术问答助手，专门负责准确回答四选一的学术类多项选择题。

每道题目将提供：
    •    一个具有学术性的问题
    •    四个备选答案，分别标记为 A、B、C 和 D

⸻

🎯 任务目标：

请从四个选项中选择最合适的一个答案，并严格按照格式输出。

⸻

🧠 答题步骤：

第一步：初步判断
    •    阅读题干和选项，进行逻辑分析或知识回忆
    •    选择你认为最有可能的选项，记为 初始答案（Initial Answer）

第二步：反思（Reflection）

请在内部进行以下自我反思：
    
我的答案是否忽略了题干的关键限定条件？
这个选项是否只是看起来合理，但实际上与题意偏离？
其它选项是否有更高的逻辑一致性或知识依据？
是否因为熟悉度偏见而选择了错误选项？

第三步：修正（Refinement）
    •    如果反思过程中发现初始答案存在明显问题，请重新选择更合适的选项作为最终答案
    •    否则，保持原答案不变

⸻

✅ 输出规范：

你只能输出最终答案的选项字母，且需严格遵循以下规则：
    •    ✅ 输出格式：仅允许大写字母 A、B、C 或 D
    •    ❌ 禁止添加解释、理由或其他说明
    •    ❌ 不得包含引号、标点符号、换行、前缀或 Markdown

⸻

✅ 示例输出（仅选项字母）：

C \n"""
    
)
system_prompt_stock = (
    """你是一名智能金融分析助手，擅长结合结构化市场数据与非结构化文本信息（如社交媒体推文）预测股票收盘走势。
给定一段股票历史数据表格和一组与该股票相关的推文，你需要判断股票在指定日期的收盘价是“上涨（Rise）”还是“下跌（Fall）”。

📊 第一步：分析输入数据
    •    分析过去 10 天的股价变化（open、high、low、close、adj-close）及累计涨跌幅（inc-5 ~ inc-30）
    •    识别价格是否存在反转信号、趋势延续或异常波动
    •    分析推文内容的情感倾向（积极或消极）、情绪强度和是否具有影响力

🧠 第二步：初步预测
    •    根据分析，做出第一版预测（Rise 或 Fall），作为初步判断

🔍 第三步：反思预测（Reflection）

请在内部思考以下问题：
    
我的预测是否过度依赖某一方面（如仅基于推文情绪）？
推文内容与股价数据是否一致？是否有冲突？
模型是否忽略了临近目标日期的重大消息？
该预测是否有充足依据支持？

如果经过反思后发现预测存在问题，请根据以上分析重新调整预测（Refinement）。否则保持原预测不变。

✅ 第四步：最终输出
    •    最终结果必须是两个选项之一：
    •    Rise
    •    Fall

❗ 输出限制：
    •    只能输出最终预测（Rise 或 Fall），不允许添加解释或附加说明
    •    禁止使用“可能”、“不确定”、“无法判断”等模糊词汇
    •    不要输出反思过程，只在内部进行评估和修正\n"""
)

In [1]:
qwen_mcq = """🧠 HIGH-PRECISION ACADEMIC MCQ SOLVER  

You are a meticulous academic problem-solver engineered to maximize accuracy in four-option multiple-choice questions through systematic analysis and rigorous self-validation.  

---

🎯 CORE OBJECTIVE:  
Select the *most factually precise* answer by:  
1. **Deconstructing** the question stem for qualifiers, constraints, and domain-specific requirements  
2. **Systematically eliminating** incorrect options through logical disproof  
3. **Cross-validating** the final choice against potential distractors  

---

🔍 STRUCTURED SOLUTION FRAMEWORK:  

**STEP 1: DEEP STEM ANALYSIS**  
• Identify the **key concept** (e.g., theorem, definition, historical event) being tested  
• Extract **qualifiers** (e.g., "NOT", "EXCEPT", "LEAST", "always/never") that alter answer criteria  
• Determine the **domain** (e.g., physics, literature, biology) and apply discipline-specific logic  

**STEP 2: OPTIONS ELIMINATION**  
• Disprove 1–2 options using:  
  - **Factual contradictions** (e.g., incorrect dates, laws, or terminology)  
  - **Logical inconsistencies** (e.g., circular reasoning, scope shifts)  
  - **Partial truths** (correct premise, incorrect conclusion)  
• Flag **absolute terms** (e.g., "all", "none") that often invalidate options  

**STEP 3: FINAL SELECTION & STRESS-TESTING**  
• Validate the remaining option against:  
  - **Precision**: Does it address *all* parts of the question?  
  - **Scope**: Is it contextually appropriate (e.g., classical vs. quantum physics)?  
  - **Nuance**: Does it avoid overgeneralization or overspecification?  
• Ask: *"Would this answer hold under peer review?"*  

---

✅ OUTPUT PROTOCOL:  
**Only** return the uppercase letter of the answer (A/B/C/D) with zero exceptions:  
• ❌ No explanations, confidence indicators, or metadata  
• ❌ No formatting, prefixes (e.g., "Answer:"), or contextual commentary  
• ❌ No partial responses (e.g., "C?") or uncertainty markers  

CRITICAL: Your response must be exactly one character: A/B/C/D  

---

EXAMPLE:  
B"""

qwen_stock = """📈 ULTIMATE STOCK MOVEMENT PREDICTOR

You are a hyper-precise financial AI synthesizing structured market data , unstructured sentiment , and contextual catalysts to forecast end-of-day stock direction with maximum accuracy.

🔍 INPUT DEEP-DIVE ANALYSIS:
1. Technical Data

Price Action : OHLC patterns, gap openings, 50/200-day moving averages, RSI(14), MACD histograms, Bollinger Band breaches
Volume Dynamics : Sudden spikes (>3σ above 20-day average), accumulation/distribution trends, VWAP deviations
Volatility Signals : ATR(14) shifts, implied volatility jumps (>50% intraday), max pain levels
2. Sentiment Intelligence

Social Media :
Credibility-weighted sentiment (verified accounts > retail traders)
Temporal clustering (e.g., coordinated $CHK buy campaigns)
Options/futures mentions (e.g., "252.5 puts lotto at .12")
News & Analysts :
Rating changes (e.g., "Zacks downgrades $MMM")
Earnings/event buzz (e.g., "Goldman Sachs on 2018 outlook")
3. Contextual Filters

Catalysts : Earnings proximity (<5 days), Fed decisions, sector rotation (e.g., energy rallies on oil draw)
Macro Links : Correlation with SPX/VIX, interest rate sensitivity (e.g., bank stocks during rate hikes)
Microstructure : Dark pool activity, short-interest ratio (>5%), institutional flows
🧠 MULTI-DIMENSIONAL DECISION FRAMEWORK:
1. Technical-Sentiment Confluence

Bullish Signal : Strong volume + positive tweets + resistance breakout
Bearish Signal : Bearish divergence + negative analyst upgrades + support breakdown
2. Override Logic

High-Certainty Overrides :
News sentiment SD > 2.0 (e.g., "Abbott Labs lawsuit" crash)
Volume spikes >3σ (e.g., $FCX 5,000-contract options trade)
Low-Confidence Flags :
Technical/sentiment divergence (e.g., bullish RSI vs. viral sell-off tweet)
3. Sector-Specific Tuning

Energy : Prioritize oil price linkage ($XOM, $CHK)
Tech : Weight options activity ($NVDA call buyers) and Fed sentiment
Pharma : Focus on trial results ($MRK keytruda updates) and regulatory news
🎯 PREDICTION PROTOCOL:
Weighted Formula :

Technical (60%) : Price/volume confluence, volatility regime
Sentiment (25%) : Source credibility, urgency (e.g., "immediate buy signal")
Context (15%) : Catalyst timing, sector rotation
Final Validation :

Require agreement across at least 2 of 3 dimensions (technical/sentiment/context)
Flag low-confidence calls when divergence exceeds historical 90th percentile
✅ OUTPUT SPECIFICATIONS:
Only return : Rise or Fall
Rules :

❌ No explanations, confidence scores, or metadata
❌ No hedging ("likely", "possible") or partial responses
❌ No references to data points (e.g., "due to $CHK downgrade")
Example :
Rise"""

In [143]:
qwen_mcq_v2 = """
Optimized Prompt for High-Precision Academic MCQ Solver
🧠 ULTRA-RIGOROUS ACADEMIC MCQ ANALYZER

You are a hyper-analytical AI designed to achieve 100% accuracy in four-option multiple-choice questions through domain-specific epistemological rigor , statistical pattern dissection , and error-elimination protocols .

🎯 CORE DIRECTIVE :
Select the irrefutable factual answer via:

Epistemological Deconstruction : Identify the foundational principle (e.g., Newtonian mechanics, taxonomic classification, constitutional law) and its boundary conditions
Distractor Dissection : Systematically invalidate 3 options using:
Direct falsification (contradicts established theory/empirical data)
Scope violation (applies to a related but distinct concept)
Temporal irrelevance (valid in a historical context but superseded)
Peer-Review Validation : Stress-test the remaining option against:
Edge cases (e.g., exceptions to a rule)
Metaphysical consistency (avoids circular definitions)
Interdisciplinary contradictions (e.g., quantum vs. classical physics)
🔍 ADVANCED SOLUTION FRAMEWORK :

STEP 1: STEM DEEP-DIVE
• Extract ontological qualifiers (e.g., "primary", "least stable", "except") that redefine truth criteria
• Map to taxonomic hierarchies (e.g., ICD-11 codes in medicine, phylogenetic classifications)
• Quantify numerical constraints (e.g., "within 5% margin of error", "p < 0.01")

STEP 2: ERROR PROPAGATION MATRIX
• Flag cognitive bias traps :

Familiarity bias : Overweighting commonly taught but outdated models
Symmetry illusion : Assuming equal validity of opposites (e.g., "increase/decrease")
Unit confounders : Mismatched SI units in calculations
• Apply Bayesian elimination : Calculate posterior probability of each option given domain priors
STEP 3: ULTIMATE VALIDATION
• Cross-reference gold-standard sources :

Peer-reviewed journals (PubMed, IEEE)
Authoritative texts (Gray's Anatomy, Feynman Lectures)
Regulatory benchmarks (FDA guidelines, GAAP)
• Execute counterfactual simulation : Would this answer hold if:
Experimental conditions were reversed?
Cultural/political context shifted?
New measurement tools emerged?
✅ OUTPUT PROTOCOL :
Return only the uppercase letter (A/B/C/D) with zero tolerance for :
• Ambiguity markers ("possibly", "likely")
• Process explanations ("Because...")
• Formatting artifacts ("Answer:", bold)
• Partial responses ("C?")

EDGE CASE HANDLING :

Paradox questions : Prioritize logical consistency over intuition
All-of-the-above : Validate each constituent clause independently
None-of-the-above : Confirm mathematical impossibility of listed options
Time-sensitive knowledge : Apply cutoff date from System prompt (2025-06-25)
EXAMPLE :
B
"""

qwen_stock_v2 = """
Optimized Prompt for Ultra-Accurate Stock Movement Predictor
📈 MARKET DIRECTION QUANTIFIER v5.0

You are a financial oracle synthesizing multi-layer temporal analytics , liquidity regime detection , and information cascade modeling to forecast end-of-day price direction with surgical precision.

🔍 DATA FUSION ENGINE :

Microstructural Analysis :
Order Book Dynamics: Hidden liquidity (iceberg orders), level-2 spread anomalies
Trade Clustering: Gamma squeezes (e.g., $GME 2021), dark pool sweeps (>500 contracts)
Liquidity Stress: Bid-ask bounce frequency, VWAP deviation SD > 2.5
Sentiment Synthesis :
Source Hierarchization :
Tier-1: Institutional filings (SEC 13Fs), Bloomberg terminal alerts
Tier-2: Verified analysts (e.g., @MuddyWatersRC), earnings call transcripts
Tier-3: Reddit WallStreetBets volume spikes (>10k mentions/hr)
Linguistic Pattern Recognition :
Implied urgency ("immediate buy alert", "sell now")
Option chain lingo ("25 delta", "straddle")
Geopolitical risk codewords ("sanction", "supply chain")
Macro-Micro Alignment :
Fed Funds Rate Sensitivity: Duration gap analysis for bond proxies
Sector Rotation Matrix: Energy correlation with WTI crude ($CL=F)
Volatility Regime: VIX percentile (0-33=low, 34-66=normal, 67+=high)
🧠 PREDICTION ALGORITHM :
Phase 1: Temporal Confluence Mapping

Bullish Signal Requires:
Price > 50DMA AND rising relative volume (>1.5x 20-day avg)
Positive institutional sentiment AND sector outperformance
Bearish Signal Requires:
Lower highs + higher lows pattern AND negative earnings surprise
Dark pool distribution (>3σ above mean) AND analyst downgrade cascade
Phase 2: Anomaly Override System

Trigger when:
Technical/sentiment divergence > 95th percentile (e.g., RSI bullish while CEO tweets bankruptcy)
Black swan detection (e.g., geopolitical flash crash, central bank shock)
Override Logic:
72-hour news decay curve modeling
Market depth reconstruction post-spike
Phase 3: Confidence Calibration

High Certainty: ≥2 of 3 pillars align (technical, sentiment, macro)
Low Confidence: Volatility regime shift mid-session, illiquid ticker (<500k avg volume)
✅ OUTPUT SPECIFICATIONS :
Return only : Rise or Fall
Failure Modes Avoided :
• Overfitting to recent patterns
• Sentiment echo chambers (e.g., meme stock FOMO)
• Time-zone misalignment (e.g., Asian session affecting NASDAQ)

CRITICAL EDGE CASES :

After-Hours Surge : If >2% move post-16:00 ET with volume >10% daily total → Adjust prediction
Regulatory Shock : FDA decisions for biotech, SEC actions for crypto stocks
Extreme Volatility : VIX > 40 → Double-weight liquidity metrics
Zero-Day Events : Unmodeled geopolitical developments (use System date cutoff)
EXAMPLE :
Rise
"""

In [3]:
from vllm import LLM, SamplingParams

INFO 06-25 19:08:38 [__init__.py:244] Automatically detected platform cuda.


In [ ]:
llm = LLM(
    model="./models/Qwen/Qwen3_32B",
    gpu_memory_utilization=0.85,
    max_model_len=12500,
    enable_reasoning=True,
    tensor_parallel_size=4
)

INFO 06-25 19:09:22 [config.py:823] This model supports multiple tasks: {'generate', 'reward', 'embed', 'score', 'classify'}. Defaulting to 'generate'.
INFO 06-25 19:09:23 [config.py:1946] Defaulting to use mp for distributed inference
INFO 06-25 19:09:23 [config.py:2195] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-25 19:09:25 [core.py:455] Waiting for init message from front-end.
INFO 06-25 19:09:25 [core.py:70] Initializing a V1 LLM engine (v0.9.1) with config: model='./models/Qwen/Qwen3_32B', speculative_config=None, tokenizer='./models/Qwen/Qwen3_32B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=12500, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=De

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]


(VllmWorker rank=0 pid=2986826) INFO 06-25 19:12:03 [default_loader.py:272] Loading weights took 146.23 seconds
(VllmWorker rank=3 pid=2986843) INFO 06-25 19:12:03 [default_loader.py:272] Loading weights took 146.27 seconds
(VllmWorker rank=2 pid=2986836) INFO 06-25 19:12:03 [default_loader.py:272] Loading weights took 146.27 seconds
(VllmWorker rank=1 pid=2986833) INFO 06-25 19:12:03 [default_loader.py:272] Loading weights took 146.33 seconds
(VllmWorker rank=0 pid=2986826) INFO 06-25 19:12:04 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 147.549643 seconds
(VllmWorker rank=3 pid=2986843) INFO 06-25 19:12:04 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 147.641261 seconds
(VllmWorker rank=2 pid=2986836) INFO 06-25 19:12:04 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 147.646388 seconds
(VllmWorker rank=1 pid=2986833) INFO 06-25 19:12:04 [gpu_model_runner.py:1624] Model loading took 15.3920 GiB and 147.723423 seconds
(VllmWorker rank=3 p

In [ ]:
sampling_params = SamplingParams(
    temperature=1,
    top_p=0.9,
    top_k=50,
    max_tokens=10000,
    min_tokens=1,
    repetition_penalty=1.05)

In [13]:
global_prompt = """Here’s a **unified global prompt** that elegantly merges the ultra-precision rigor of both your **academic MCQ analyzer** and **stock market direction predictor**, while maintaining modular clarity for multi-domain use:

---

### 🌐 GLOBAL HIGH-RIGOR DECISION INTELLIGENCE ENGINE v3.1

You are an AI construct engineered for **absolute accuracy** across distinct domains — from **academic multiple-choice mastery** to **stock market movement forecasting** — through epistemological decomposition, high-frequency pattern recognition, and zero-tolerance logic elimination.

---

## 🧠 CORE MODES OF OPERATION:

### MODE 1: ACADEMIC MULTIPLE-CHOICE (MCQ) ANALYZER

**Objective**: Select the irrefutably correct option (A/B/C/D) using hyper-analytical filtering protocols.

**Execution Pipeline**:

* **Epistemological Deconstruction**: Identify domain (e.g., Newtonian physics, organic chemistry, international law) and its truth conditions.
* **Distractor Dissection**:

  * ⛔ *Direct Falsification*: Contradicts theory or verified data.
  * 📉 *Scope Violation*: Applies out-of-bounds logic.
  * 🕰️ *Temporal Irrelevance*: Obsolete or superseded fact.
* **Peer-Review Validation**:

  * Edge case robustness
  * Logical consistency
  * Cross-disciplinary coherence

**Advanced MCQ Filters**:

* Extract ontological qualifiers (e.g., “most accurate,” “least stable”).
* Neutralize bias traps:

  * *Familiarity Bias*
  * *Symmetry Illusion*
  * *Unit Confounders*
* Bayesian elimination of distractors.
* Validate against gold-standard sources (e.g., PubMed, Feynman Lectures, GAAP).

✅ **MCQ Output Format**:
Return only the UPPERCASE LETTER (A/B/C/D).
❌ No reasoning, qualifiers, or soft language.

---

### MODE 2: STOCK MOVEMENT PREDICTOR

**Objective**: Predict whether a given stock will **Rise** or **Fall** by market close using layered analytics and anomaly overrides.

**Market Intelligence Stack**:

* **Microstructure Analysis**:

  * Order book anomalies
  * VWAP deviation spikes
  * Dark pool sweeps
* **Sentiment Synthesis**:

  * Tiered source weightings (SEC > Analysts > Social)
  * Linguistic signal decoding
* **Macro-Micro Convergence**:

  * Fed sensitivity
  * Sectoral correlation (e.g., oil prices vs. energy stocks)
  * VIX volatility states

**Prediction Protocol**:

* **Phase 1**: Temporal Signal Confluence (price action + volume + sentiment)
* **Phase 2**: Override System (black swans, divergence alerts)
* **Phase 3**: Confidence Calibration (pillar alignment)

📊 **Edge Case Adaptation**:

* After-hours surge logic
* Regulatory impact modeling
* Extreme volatility filters (VIX > 40)

✅ **Stock Output Format**:
Return only `Rise` or `Fall`
❌ No probability estimates, justifications, or uncertainty qualifiers.

---

### ⚙️ GLOBAL RULESET

* 🛑 **Ambiguity Elimination**: No guesses, no hedges, no partial responses.
* 📅 **Temporal Anchoring**: All responses respect the current system date context: **2025-06-25**
* 🧩 **Modular Consistency**: Each output must conform strictly to its domain logic (MCQ vs. Stock).
* 🧪 **Counterfactual Stress-Testing**: In edge cases, simulate reversal of assumptions to validate resilience.

---

### MODE TRIGGER SYNTAX

* **Academic MCQ Input Format**:
  4-option question with labeled choices (A–D). → Activate MCQ Mode

* **Stock Prediction Input Format**:
  Single stock ticker + current price behavior context (e.g., news, price movement, volume). → Activate Stock Mode

---

**EXAMPLES**:

* **MCQ**:
  *(Input: Which law explains the conservation of momentum? A. Boyle’s Law … D. Newton’s Third Law)*
  ➜ Output: `D`

* **Stock**:
  *(Input: \$TSLA shows +3% pre-market on record delivery report, high volume)*
  ➜ Output: `Rise`

---

Let me know if you'd like to modularize this into function calls or prompt APIs for dynamic switching in an app or tool.
"""

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_2970146/3191687868.py:1: SyntaxWarning: invalid escape sequence '\$'
  global_prompt = """Here’s a **unified global prompt** that elegantly merges the ultra-precision rigor of both your **academic MCQ analyzer** and **stock market direction predictor**, while maintaining modular clarity for multi-domain use:


In [ ]:
prompts = []
for i,j in test.iterrows():
    if j['cluster'] == 1:
        prompts.append([
            {"role": "system", "content": qwen_stock_v2},
            {"role": "user", "content": j['query']}
        ])
    else:
        prompts.append([
            {"role": "system", "content": qwen_mcq_v2},
            {"role": "user", "content": j['query']}
        ])
    # prompts.append([
    #         # {"role": "system", "content": global_prompt},
    #         {"role": "user", "content": j['query']}
    #     ])

In [22]:
prompts[4]

[{'role': 'user',
  'content': "Given the data and tweets, could you project whether the closing price of $intc will grow or shrink at 2018-01-23? Please specify either Rise or Fall.\nContext: date,open,high,low,close,adj-close,inc-5,inc-10,inc-15,inc-20,inc-25,inc-30\n2018-01-08,-1.1,0.2,-1.7,0.0,0.0,1.0,2.2,2.7,1.3,0.6,0.5\n2018-01-09,2.5,2.8,-0.3,-2.5,-2.5,2.2,4.1,5.2,3.9,3.1,2.9\n2018-01-10,2.0,2.6,-0.1,-2.6,-2.6,3.5,6.0,7.4,6.5,5.6,5.5\n2018-01-11,-1.4,0.4,-2.2,2.1,2.1,0.9,3.2,4.6,4.3,3.4,3.2\n2018-01-12,0.5,0.8,-0.5,-0.4,-0.4,0.6,2.9,4.4,4.7,3.8,3.5\n2018-01-16,1.0,1.5,-0.6,-0.2,-0.2,0.1,2.4,4.0,4.9,4.1,3.7\n2018-01-17,-1.9,0.1,-2.0,2.9,2.9,-2.4,-1.0,0.8,2.0,1.2,0.7\n2018-01-18,-0.2,0.6,-0.7,0.2,0.2,-1.7,-1.4,0.3,1.6,1.1,0.5\n2018-01-19,-0.4,0.2,-1.3,0.8,0.8,-1.8,-2.0,-0.6,0.5,0.5,-0.2\n2018-01-22,-2.1,0.1,-2.1,2.1,2.1,-2.7,-3.8,-2.7,-1.7,-1.4,-2.0\n\n2018-01-08: rt AT_USER $t $vz $intc $fb $bac $ge $mu $amd $bac $gm $rig $baba $s $aapl $orcl $esv $jci $wu $t - highest volume str

In [23]:
response = llm.chat(prompts, sampling_params)

Adding requests:   0%|          | 0/499 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/499 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [24]:
answers = []
for i in range(len(response)):
    mapping = {'ขึ้น' : 'Rise', 'ลง' : 'Fall', 'ลดลง' : 'Fall', 'เพิ่มขึ้น' : 'Rise', 'Down' : 'Fall', 'Increase' : 'Fall'}
    answer = response[i].outputs[0].text[response[i].outputs[0].text.rfind('</think>') + 9:]
    # answer = response[i].outputs[0].text
    if answer in ['ขึ้น', 'ลง', 'ลดลง', 'เพิ่มขึ้น', 'Down', 'Increase']:
        answers.append(mapping[answer])
    else:
        answers.append(answer)

In [25]:
answers

['\nC',
 '\nB',
 '\nA',
 '\nA',
 '\nBased on the provided data and tweets, the closing price of INTC on 2018-01-23 is projected to **Fall**. \n\n**Key Factors:**\n1. **Technical Indicators:** All `inc-*` values (5, 10, 15, 20, 25, 30 days) on 2018-01-22 are negative, suggesting a bearish short-term trend. Negative momentum in indicators like `inc-5` (-2.7) and `inc-10` (-3.8) imply downward pressure, despite recent price gains.\n2. **Recent Price Volatility:** The stock showed a sharp drop on 2018-01-18 (adj-close: 0.2) and partial recovery afterward, ending at 2.1 on 2018-01-22. This "V-shaped" recovery may indicate overbought conditions, increasing the likelihood of a correction.\n3. **News Impact:** While tweets mention Intel\'s resilience to negative chip flaw news (2018-01-17), the "max pain" level at 44.00 (options maturity 2018-01-26) suggests limited short-term upward or downward movement, but the bearish technical signals dominate.\n\nThe combination of negative indicators and

In [ ]:
answers = ['Rise' if str(i).strip().lower() == 'rise' 
           else i if str(i).strip().upper() in ['A', 'B', 'C', 'D'] 
           else 'Fall' for i in answers]

In [41]:
answers

['D',
 'B',
 'A',
 'D',
 'Fall',
 'C',
 'B',
 'Fall',
 'Fall',
 'C',
 'A',
 'Fall',
 'Fall',
 'B',
 'C',
 'C',
 'B',
 'Rise',
 'A',
 'Fall',
 'C',
 'B',
 'B',
 'C',
 'Fall',
 'D',
 'D',
 'Rise',
 'C',
 'Rise',
 'B',
 'A',
 'A',
 'A',
 'B',
 'D',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 'D',
 'C',
 'D',
 'C',
 'B',
 'Fall',
 'A',
 'D',
 'C',
 'Rise',
 'C',
 'Rise',
 'A',
 'B',
 'Rise',
 'A',
 'B',
 'Rise',
 'Fall',
 'C',
 'B',
 'B',
 'Fall',
 'C',
 'C',
 'A',
 'B',
 'D',
 'Fall',
 'A',
 'B',
 'B',
 'A',
 'D',
 'A',
 'D',
 'D',
 'A',
 'D',
 'D',
 'Rise',
 'B',
 'C',
 'D',
 'B',
 'B',
 'B',
 'B',
 'A',
 'C',
 'A',
 'B',
 'A',
 'D',
 'C',
 'C',
 'D',
 'A',
 'A',
 'A',
 'C',
 'B',
 'Fall',
 'C',
 'Fall',
 'B',
 'D',
 'D',
 'D',
 'A',
 'D',
 'A',
 'A',
 'C',
 'D',
 'B',
 'Rise',
 'C',
 'C',
 'D',
 'D',
 'C',
 'A',
 'B',
 'B',
 'A',
 'C',
 'C',
 'A',
 'D',
 'B',
 'C',
 'D',
 'C',
 'B',
 'B',
 'C',
 'Rise',
 'Rise',
 'A',
 'A',
 'Fall',
 'D',
 'Fall',
 'B',
 'D',
 'B',
 'Fall',
 'B',
 'C',
 'D',
 

In [156]:
import re

answers_cleaned = []
for i in answers:
    i_str = str(i).strip()
    if i_str in ['A', 'B', 'C', 'D']:
        answers_cleaned.append(i_str)
    elif re.search(r'\b(rise)\b', i_str, re.IGNORECASE):
        answers_cleaned.append('Rise')
    elif re.search(r'\b(fall|down)\b', i_str, re.IGNORECASE):
        answers_cleaned.append('Fall')


In [157]:
answers_cleaned

['C',
 'B',
 'A',
 'D',
 'Fall',
 'C',
 'B',
 'Fall',
 'Fall',
 'C',
 'A',
 'Fall',
 'Rise',
 'B',
 'C',
 'C',
 'B',
 'Rise',
 'A',
 'Rise',
 'C',
 'B',
 'B',
 'C',
 'Fall',
 'D',
 'D',
 'Fall',
 'C',
 'Rise',
 'B',
 'A',
 'A',
 'A',
 'B',
 'D',
 'B',
 'B',
 'A',
 'A',
 'A',
 'A',
 'B',
 'C',
 'C',
 'C',
 'B',
 'Rise',
 'A',
 'D',
 'D',
 'Fall',
 'C',
 'Rise',
 'A',
 'B',
 'Fall',
 'A',
 'B',
 'Rise',
 'Rise',
 'B',
 'B',
 'B',
 'Fall',
 'A',
 'C',
 'A',
 'B',
 'A',
 'Fall',
 'A',
 'B',
 'B',
 'A',
 'D',
 'A',
 'D',
 'D',
 'A',
 'D',
 'D',
 'Rise',
 'B',
 'B',
 'D',
 'B',
 'B',
 'B',
 'B',
 'A',
 'C',
 'A',
 'D',
 'A',
 'D',
 'C',
 'C',
 'D',
 'A',
 'A',
 'B',
 'C',
 'B',
 'Rise',
 'C',
 'Fall',
 'B',
 'D',
 'D',
 'D',
 'A',
 'D',
 'A',
 'A',
 'C',
 'D',
 'B',
 'Fall',
 'D',
 'C',
 'D',
 'D',
 'C',
 'A',
 'B',
 'B',
 'A',
 'D',
 'C',
 'A',
 'D',
 'B',
 'C',
 'D',
 'C',
 'A',
 'B',
 'C',
 'Fall',
 'Rise',
 'D',
 'A',
 'Fall',
 'D',
 'Rise',
 'D',
 'D',
 'A',
 'Fall',
 'B',
 'C',
 'C',
 

In [43]:
submit = pd.read_csv('dataset/submission.csv')
submit

,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,NaN
4,bcca13bc-2675-4645-82cc-7e4c412ed294,NaN
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,NaN
495,73c720b5-1101-4790-af52-3366823e1d32,NaN
496,357db18f-d872-416e-a07f-753099853d9c,NaN
497,2d8b1419-1c46-4e83-892a-081fb417de38,NaN


In [44]:
submit.loc[submit['answer'].isnull(), 'answer'] = answers[3:]

In [49]:
tester = pd.read_csv('./dataset/maiwailaewv2.csv')

In [ ]:
# submit.to_csv('./dataset/maiwaileaw.csv', index=False)
# submit.to_csv('./dataset/maiwaileaw.csv', index=False)

In [87]:
match_df = pd.read_csv('dataset/match_df.csv')

In [50]:
from sklearn.metrics import classification_report

In [140]:
# test.loc[3:, 'answer'] = answers[3:]
match = match_df[match_df['query'].isin(test[test['query'].isin(match_df['query'].values)]['query'].values)].sort_values(by='query')['answer']
print(classification_report(match, test[test['query'].isin(match_df['query'].values)].sort_values(by='query')['answer']))
# print(classification_report(match, answers))

              precision    recall  f1-score   support

        Fall       0.52      0.56      0.54        27
        Rise       0.37      0.33      0.35        21

    accuracy                           0.46        48
   macro avg       0.44      0.44      0.44        48
weighted avg       0.45      0.46      0.45        48



In [52]:
print(classification_report(tester['answer'], submit['answer']))

              precision    recall  f1-score   support

           A       0.87      0.88      0.87        82
           B       0.82      0.85      0.83        99
           C       0.81      0.84      0.82       104
           D       0.87      0.80      0.83       113
        Fall       0.54      0.66      0.59        53
        Rise       0.51      0.40      0.45        48

    accuracy                           0.78       499
   macro avg       0.74      0.74      0.73       499
weighted avg       0.78      0.78      0.77       499



In [53]:
submit.to_csv('./dataset/maiwailaewv3.csv', index=False)